In [1]:
import pandas as pd
import numpy as np
import os
import datetime 
import matplotlib.pyplot as plt
from scipy import linalg

<b>Pulling Informtion from CSV Files</b>

In [2]:
#Pull the valid stock information
slist = []
filelist = os.listdir('c:/jmmbdevwsp/stockcsv/')
for filename in filelist:
    slist.append(pd.read_csv('c:/jmmbdevwsp/stockcsv/'+filename))
    
frame = pd.concat(slist, axis = 0, ignore_index = True)
frame = frame.sort_values(by='Date')
frame.head()

,Symbol,Date,52 Week High,52 Week Low,Previous Year Div,Current Year Div,Volume (non block),Today High,Today Low,Last Traded,Close Price,Price Change,Closing Bid,Closing Ask
161892,CWJ,2008-01-02,2.0,0.13,NaN,NaN,166310.0,0.67,0.66,0.66,0.66,0.02,0.66,0.68
161908,SVL,2008-01-02,5.0,1.60,NaN,NaN,32000.0,2.40,2.40,2.40,2.40,-0.43,2.38,2.40
161907,SLJ,2008-01-02,11.3,6.60,NaN,NaN,20189.0,8.15,8.00,8.00,8.00,0.00,8.00,8.15
161906,SGJ,2008-01-02,34.0,16.15,NaN,NaN,41066.0,21.05,21.05,21.05,21.05,-0.25,21.05,21.45
161905,RJR,2008-01-02,5.6,1.00,NaN,NaN,55340.0,3.30,3.27,3.27,3.27,-0.13,3.30,3.63


In [3]:
#Pull the list of delisted to remove from stock frame
framevalid = pd.read_csv('c:/jmmbdevwsp/stockdelist/stockvalid.csv')
framevalid = framevalid.rename(columns={'Instrument Code': 'Symbol'})
framevalid.head()

,Name,Symbol,Currency,Sector,Type,Website
0,138 STUDENT LIVING JAMAICA LIMITED,138SL,JMD,OTHER,ORDINARY,-
1,138 STUDENT LIVING JAMAICA LIMITED VARIABLE PR...,138SLVR,JMD,OTHER,PREFERENCE,-
2,1834 INVESTMENTS LIMITED,1834,JMD,COMMUNICATIONS,ORDINARY,Website
3,BARITA INVESTMENTS LIMITED,BIL,JMD,FINANCE,ORDINARY,Website
4,BERGER PAINTS JAMAICA LTD.,BRG,JMD,MANUFACTURING,ORDINARY,Website


In [4]:
slist = []
slist.append(pd.read_csv('c:/jmmbdevwsp/stocksplit/corporate-actions (1).csv'))
slist.append(pd.read_csv('c:/jmmbdevwsp/stocksplit/corporate-actions (2).csv'))
df_split = pd.concat(slist, axis = 0, ignore_index = True)
df_split.head()

,Symbol,Record Date,Action,Dividend Amount,Ex-Date,Payment Date
0,EPLY,12/4/2018,NaN,0.037200,12/3/2018,12/18/2018
1,1834,12/3/2018,NaN,0.100000,11/30/2018,12/19/2018
2,JMMBGL7.50,11/30/2018,NaN,0.004635,11/29/2018,12/14/2018
3,JMMBGL7.25,11/30/2018,NaN,0.006953,11/29/2018,12/14/2018
4,SIL,11/27/2018,Stock Split,5.000000,11/26/2018,NaN


<html><body><b>Time for Clean Up</b></body></html>

In [5]:
framevalid = framevalid[framevalid.Type == "ORDINARY"]
framevalid.head()

,Name,Symbol,Currency,Sector,Type,Website
0,138 STUDENT LIVING JAMAICA LIMITED,138SL,JMD,OTHER,ORDINARY,-
2,1834 INVESTMENTS LIMITED,1834,JMD,COMMUNICATIONS,ORDINARY,Website
3,BARITA INVESTMENTS LIMITED,BIL,JMD,FINANCE,ORDINARY,Website
4,BERGER PAINTS JAMAICA LTD.,BRG,JMD,MANUFACTURING,ORDINARY,Website
6,CARIBBEAN CEMENT COMPANY LTD.,CCC,JMD,MANUFACTURING,ORDINARY,Website


In [6]:
#Remove unnecessary columns
frame = frame.drop(['52 Week High','52 Week Low','Current Year Div','Today High','Today Low','Last Traded','Price Change','Closing Bid','Closing Ask','Previous Year Div','Volume (non block)'],axis=1)
frame.head()

,Symbol,Date,Close Price
161892,CWJ,2008-01-02,0.66
161908,SVL,2008-01-02,2.40
161907,SLJ,2008-01-02,8.00
161906,SGJ,2008-01-02,21.05
161905,RJR,2008-01-02,3.27


In [7]:
#filter for only the valid stocks
frame = frame[frame["Symbol"].isin(framevalid["Symbol"].tolist())]
frame.head()

,Symbol,Date,Close Price
161908,SVL,2008-01-02,2.40
161906,SGJ,2008-01-02,21.05
161905,RJR,2008-01-02,3.27
161900,MIL,2008-01-02,4.50
161904,PJAM,2008-01-02,47.00


In [8]:

frame['Date'] = pd.to_datetime(frame['Date'])
frame.head()

,Symbol,Date,Close Price
161908,SVL,2008-01-02,2.40
161906,SGJ,2008-01-02,21.05
161905,RJR,2008-01-02,3.27
161900,MIL,2008-01-02,4.50
161904,PJAM,2008-01-02,47.00


In [9]:
df_split["Action"] = df_split["Action"].str.strip()
df_split = df_split[df_split["Action"]=='Stock Split']
df_split = df_split.drop(['Record Date','Action','Payment Date'],axis=1)
df_split['Ex-Date'] = pd.to_datetime(df_split['Ex-Date'])
df_split = df_split.dropna(axis=0,how='any')
df_split

,Symbol,Dividend Amount,Ex-Date
4,SIL,5.0,2018-11-26
57,DTL,10.0,2018-09-05
65,BPOW,10.0,2018-08-28
269,CAR,10.0,2017-09-18
323,PULS,5.0,2017-07-03
336,JETCON,4.0,2017-06-22
355,KEX,5.0,2017-05-31
362,KPREIT,2.0,2017-05-19
384,JAMT,2.0,2017-04-18
409,AMG,5.0,2017-03-10


In [10]:
#Fix the stock Split Issues
frame['slit_weight'] = np.nan
frame.head()

,Symbol,Date,Close Price,slit_weight
161908,SVL,2008-01-02,2.40,NaN
161906,SGJ,2008-01-02,21.05,NaN
161905,RJR,2008-01-02,3.27,NaN
161900,MIL,2008-01-02,4.50,NaN
161904,PJAM,2008-01-02,47.00,NaN


In [11]:
#Remove Duplicated stock price
frame = frame.drop_duplicates(subset = ["Symbol","Date"], keep=False)

In [12]:
#Can be used to check for duplication but will raise error of no duplication 
#pd.concat(g for _, g in frame.groupby(["Symbol","Date"]) if len(g) > 1)

In [13]:
gp = frame.groupby("Symbol")

In [18]:
stklist = [gp.get_group(x) for x in gp.groups]

In [21]:
for idx, stk in enumerate(stklist):
    stklist[idx] = stklist[idx].set_index('Date')    

stklist

           Symbol  Close Price  slit_weight
Date                                       
2017-03-10    AMG         4.87          5.0
           Symbol  Close Price  slit_weight
Date                                       
2018-08-28   BPOW          4.7         10.0
           Symbol  Close Price  slit_weight
Date                                       
2017-09-18    CAR        10.87         10.0
           Symbol  Close Price  slit_weight
Date                                       
2018-09-05    DTL         2.65         10.0
           Symbol  Close Price  slit_weight
Date                                       
2017-04-18   JAMT         3.67          2.0
            Symbol  Close Price  slit_weight
Date                                        
2017-06-22  JETCON         5.66          4.0
           Symbol  Close Price  slit_weight
Date                                       
2017-05-31    KEX         15.0          5.0
            Symbol  Close Price  slit_weight
Date                        

[]

In [ ]:
dividend = []
for idx, stk in enumerate(stklist):    
    symbol = stk['Symbol'][0]
    if not df_split[(df_split.Symbol == symbol)].empty:
        split_s = df_split[(df_split.Symbol == symbol)]                
        stk.at[split_s['Ex-Date'].iloc[0],'slit_weight'] = split_s['Dividend Amount'].iloc[0]
        print(stk[stk.index.isin(split_s['Ex-Date'].tolist())])                
        stklist[idx] = stk
        

In [ ]:
stkframe = pd.DataFrame()
for stk in stklist:
    stkframe[stk['Symbol'][0]] = stk['Close Price']

In [ ]:
stkframe.head()

In [ ]:
stkframe.tail()

In [ ]:
#filter the data for the 01 January 2018 to 03 December, 2018
stkframe=stkframe['2017-01-01': '2018-12-03']

In [ ]:
stkframe.head()

In [ ]:
stkframe.tail()

<b>Time for Some Financial Analysis</b>

In [ ]:
rate_return = (stkframe/stkframe.shift(1))-1
rate_return.head()

In [ ]:
rate_return = rate_return.replace([np.inf, -np.inf], np.nan)
rate_return.head()

In [ ]:
#Sample of rate of return
rate_return['AMG'].plot()
plt.figure(figsize=(30,15), dpi=80, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 5})
plt.show()

In [ ]:
df = pd.DataFrame()

df['closing'] = stkframe['AMG']
#df['rmean_3'] = df['closing'].rolling( 3 ).mean()
#df['rmean_6'] = df['closing'].rolling( 6 ).mean()
df['rmean_12'] = df['closing'].rolling( 12 ).mean()
plt.figure(figsize=(30,15), dpi=80, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 22})
plt.plot(df)

In [ ]:
stkframe['2018-08-01':'2018-08-30']['BPOW']

In [ ]:
#Calculate the Average Rate of Return
avg_returns = round(rate_return.mean() * len(rate_return),5) * 100
# Drop all records with no application
avg_returns = avg_returns.dropna()
# Drop all record with 0 as rate of return
avg_returns = avg_returns.drop(avg_returns[avg_returns == 0].index)
avg_returns.sort_values(ascending=False)

In [ ]:
#find the stocks that have split
avg_returns[avg_returns <= 0].sort_values()